In [4]:
!python --version

Python 3.11.5


In [1]:
!pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1

ERROR: To modify pip, please run the following command:
C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42
   ------------- -------------------------- 0.7/2.1 MB 13.8 kB/s eta 0:01:42

In [2]:
!pip show pip

Name: pip
Version: 23.2.1
Summary: The PyPA recommended tool for installing Python packages.
Home-page: https://pip.pypa.io/
Author: The pip developers
Author-email: distutils-sig@python.org
License: MIT
Location: C:\ProgramData\anaconda3\Lib\site-packages
Requires: 
Required-by: 


In [ ]:
!pip install tensorflow

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load the eeg_data and participants info datasets

eeg_df = pd.read_csv('eeg_data.csv')

In [ ]:
info_df = pd.read_csv('participants_info.csv')

In [ ]:
eeg_df.head()

In [ ]:
# remove the unwanted Unnamed column

eeg_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
eeg_df

In [ ]:
info_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
info_df

In [ ]:
model_df = info_df.merge(eeg_df, on='ParticipantID')

In [ ]:
model_df.head()

In [ ]:
model_df.info()

In [ ]:
model_df.drop(columns=['firstname', "Date of Birth", 'surname', 'lastSessionID'], inplace=True)

In [ ]:
len(model_df.select_dtypes('object').columns)

In [ ]:
categoricals = model_df.select_dtypes('object').columns
categoricals

In [ ]:
model_df

In [ ]:
for x in categoricals:
    print(model_df[x].unique())

In [ ]:
for categorical in categoricals:
    uniques = model_df[categorical].unique()
    model_df[categorical] = model_df[categorical].apply(lambda x: 1 if x == uniques[0] else 0)
    
# control is 1 Patient is 0

In [ ]:
model_df

In [ ]:
model_df['sex'].unique()

In [ ]:
print(model_df.select_dtypes('object').columns)

# no more non-numeric columns

## EDA

In [ ]:
features = model_df.drop('category', axis=1).copy()

In [ ]:
num_features = len(features.columns)
print('Features: ', num_features)

In [ ]:
cat_features = features[['language', 'sex', 'age']]

In [ ]:
'Categorical Features: ', len(cat_features.columns)

In [ ]:
cont_features = features.drop(columns=['language', 'sex', 'ParticipantID', 'age'], axis=1)

In [ ]:
'Continuous Features: ', len(cont_features.columns)

In [ ]:
model_df.dtypes

## Univariate Analysis

In [ ]:
features[cont_features.columns].plot(kind='box', figsize=(15, 15), subplots=True, layout=(7, 5))
plt.show()

In [ ]:
features[cont_features.columns].plot(kind='hist', bins=25, figsize=(15, 12), subplots=True, layout=(6, 5))
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
for index, feature in enumerate(cat_features):
    plt.subplot(1, 5, index + 1)
    features[feature].value_counts().plot(kind='pie')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
model_df['category'].value_counts().plot(kind='pie', autopct='%.1f%%')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
sns.pairplot(features[cont_features.columns])
plt.show()

In [ ]:

corr = model_df.corr()

plt.figure(figsize=(30, 20))
sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
plt.show()

In [ ]:
# drop the participant ID since the model is not meant for specific patients
model_df = model_df.drop(columns=['ParticipantID'], axis=1)

In [ ]:
y = model_df['category'].copy()
X = model_df.drop('category', axis=1).copy()

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [ ]:
model_df

## Training

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

batch_size = 32
epochs = 50

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

In [ ]:
plt.figure(figsize=(16,10))

plt.plot(range(epochs), history.history['loss', label='Training Loss'])
plt.plot(range(epochs), history.history['val_loss', label='Validation Loss'])

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Time')
plt.legend()

plt.show

## Results

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_true = np.array(y_test)

y_pred = np.squeeze(model.predict(X_test))
y_pred = np.array(y_pred >= 0.5, dtype=np.int)

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(4, 4))

sns.heatmap(cm, annot=True, fmt='g', vmin=0, cbar=False)

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

plt.show()

In [ ]:
print(classification_report(y_true, y_pred))